In [1]:
# importing libraries
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocessing test and validation images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# build model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

C:\Users\hp\AppData\Local\Temp\ipykernel_15140\3198193097.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 414s 919ms/step - loss: 1.8033 - accuracy: 0.2611 - val_loss: 1.7157 - val_accuracy: 0.3496
Epoch 2/50
448/448 [==============================] - 337s 752ms/step - loss: 1.6272 - accuracy: 0.3687 - val_loss: 1.5323 - val_accuracy: 0.4148
Epoch 3/50
448/448 [==============================] - 296s 662ms/step - loss: 1.5194 - accuracy: 0.4155 - val_loss: 1.4458 - val_accuracy: 0.4477
Epoch 4/50
448/448 [==============================] - 313s 698ms/step - loss: 1.4520 - accuracy: 0.4413 - val_loss: 1.3909 - val_accuracy: 0.4768
Epoch 5/50
448/448 [==============================] - 303s 676ms/step - loss: 1.3944 - accuracy: 0.4702 - val_loss: 1.3500 - val_accuracy: 0.4844
Epoch 6/50
448/448 [==============================] - 329s 735ms/step - loss: 1.3442 - accuracy: 0.4883 - val_loss: 1.3125 - val_accuracy: 0.5000
Epoch 7/50
448/448 [==============================] - 336s 751ms/step - loss: 1.2997 - accuracy: 0.5089 - val_loss: 1.2757 -

In [6]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')